## 搭建ResNet

In [1]:
import pickle
import os
import numpy as np
CIFAR_DIR = "../data/cifar-10-batches-py"

In [2]:
for file in os.listdir(CIFAR_DIR):
    print(file.title())
print(os.path.join(CIFAR_DIR,os.listdir(CIFAR_DIR)[1].title()))

Batches.Meta
Data_Batch_1
Data_Batch_2
Data_Batch_3
Data_Batch_4
Data_Batch_5
Readme.Html
Test_Batch
../data/cifar-10-batches-py\Data_Batch_1


In [3]:
def load_single_data(filename):
    """读取单个数据集中的数据"""
    with open(filename, 'rb') as one_file:
        batch_data = pickle.load(one_file, encoding = "bytes")
        return batch_data[b'data'], batch_data[b'labels']
    
# 数据读取处理
class CIFAR10_Read:
    def __init__(self, filenames, need_shuffle):
        """从文件列表中读取数据"""
        all_data = []
        all_labels = []
        for filename in filenames:
            """将读取的数据存入数组中"""
            data, labels = load_single_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(all_labels)
        print(self._data.shape, self._labels.shape)
        self._num_examples = len(self._data)
        self._need_shuffle  = need_shuffle
        self._index = 0
        if need_shuffle:
            self.shuffle_data()
            
    def shuffle_data(self):
        """将数据进行乱序处理"""
        p_index = np.random.permutation(self._num_examples)
        self._data = self._data[p_index]
        self._labels = self._labels[p_index]
    
    def next_batch(self, batch_size):
        """读取下一组数据集"""
        if batch_size > self._num_examples:
            raise Exception("batch size is larger than examples' num")
        end_index = self._index + batch_size
        if end_index > self._num_examples:
            if self._need_shuffle:
                """如果当前数据读取完毕，且如果可以对数据进行重排列，则重新组合数据集"""
                self.shuffle_data()
                self._index = 0
                end_index = batch_size
            else:
                raise Exception("there is no more data")
        batch_data, batch_labels = self._data[self._index:end_index], self._labels[self._index:end_index] 
        self._index = end_index
        return batch_data, batch_labels
        

In [4]:
from tensorflow import compat as tf_compat
tf = tf_compat.v1
tf.disable_v2_behavior()

f:\anaconda\envs\tf_2g\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
non-resource variables are not supported in the long term


## 搭建ResNet的计算图

In [5]:
# 搭建残差网络块
def residual_block(input_data, output_channel):
    input_channel = input_data.get_shape().as_list()[-1] # 获取输入通道数
    if input_channel * 2 == output_channel:
        """如果输出通道数等于输入通道数的两倍，则需要做降采样过程"""
        increase_dim = True
        strides = (2, 2)
    elif input_channel == output_channel:
        """通道数一样"""
        increase_dim = False
        strides = (1, 1)
    else:
        raise Exception("input channels can't match output channels")
    conv1 = tf.layers.conv2d(input_data,
                             output_channel,
                             (3, 3),
                             strides,
                             padding = "same",
                             activation = tf.nn.relu,
                             name = "conv1")
    conv2 = tf.layers.conv2d(conv1,
                             output_channel,
                             (3, 3),
                             (1, 1),
                             padding = "same",
                             activation = tf.nn.relu,
                             name = "conv2")
    if increase_dim:
        # 将输入数据进行降采样
        pooled_input = tf.layers.average_pooling2d(input_data,
                                                   (2, 2),
                                                   (2, 2),
                                                   padding = "valid",
                                                   name = "pool")
        # 将降采样后输入的通道数翻倍
        pad_input = tf.pad(pooled_input, 
                          [[0, 0],
                           [0, 0],
                           [0, 0],
                           [input_channel // 2, input_channel // 2]])
    else:
        pad_input = input_data
    output_data = conv2 + pad_input
    return output_data

def res_net(input_images, classes, residual_block_list, filter_base):
    # 获取输入图像的大小
    input_size = input_images.get_shape().as_list()[1:]
    # 每一个卷积层
    layers = []    
    with tf.variable_scope("conv0"):
        conv0 = tf.layers.conv2d(input_images,
                                filter_base,
                                (3, 3),
                                (1, 1),
                                padding = "same",
                                name = "conv0")
        layers.append(conv0)
    # 降采样的次数
    num_sample = len(residual_block_list)
    for sample_id in range(num_sample):
        for block_id in range(residual_block_list[sample_id]):
            with tf.variable_scope("conv%d_%d" % (sample_id, block_id)):
                conv_x = residual_block(layers[-1],filter_base*(2**sample_id))
                layers.append(conv_x)
    multiplier = 2**(num_sample - 1)
    assert layers[-1].get_shape().as_list()[1:] == [input_size[0] // multiplier, input_size[1] // multiplier, filter_base * multiplier]
    print(layers[-1].get_shape().as_list()[-1])
    # 最后直接输出结果
    with tf.variable_scope("fc"):
        global_pool = tf.reduce_mean(layers[-1], [1, 2])
        logits = tf.layers.dense(global_pool, classes)
        layers.append(logits)
    return layers[-1]
                

In [6]:
# 输入数据
x = tf.placeholder(tf.float32, [None, 3072])
# 输出标签
y = tf.placeholder(tf.int64, [None])
# 将一维数据转为三维数据
x_image = tf.reshape(x, [-1, 3, 32, 32])
# 将原始图像的维度进行转换，即转置
x_image = tf.transpose(x_image, perm = [0, 2, 3, 1])

y_ = res_net(x_image, 10, [2, 3, 2], 32)

loss = tf.losses.sparse_softmax_cross_entropy(labels = y, logits = y_)
# # 进行概率映射
# p_y = tf.nn.softmax(y_)
# # 使用one_hot编码
# y_one_hot = tf.one_hot(y, 10, dtype = tf.float32)
# # 计算损失函数
# loss = tf.reduce_mean(tf.square(y_one_hot - p_y))
# 从计算的结果中，获取最大值的索引作为预测的结果值
predict = tf.argmax(y_, 1)
# 计算平均正确率
correct_predict = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float64))
    
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.AveragePooling2D instead.
128
Instructions for updating:
Use keras.layers.Dense instead.


In [ ]:
initializer =  tf.global_variables_initializer()
batch_steps = 100000
batch_size = 20
test_steps = int(2000 / batch_size)
train_filenames = [os.path.join(CIFAR_DIR, "Data_Batch_%d" % i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, "Test_Batch")]
print(train_filenames)
train_data_reader = CIFAR10_Read(train_filenames, True)
def print_log(step,  acc, current_loss_val = None, is_test = False):
    if is_test:
        print("[Test] Step %d, acc: %4.5f." %(step, acc))
    else:
        print("[Train] Step %d, loss: %4.5f, acc: %4.5f." %(step, current_loss_val, acc))
        
    
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
# 初始化所有的变量数据
    sess.run(initializer)
    for one_train_step in range(batch_steps):
        one_batch_data, one_batch_labels = train_data_reader.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run([loss, accuracy, train_op],
                                        feed_dict={
                                            x: one_batch_data,
                                            y: one_batch_labels
                                        })
        if (one_train_step + 1) % 500 == 0:
            print_log(step = one_train_step+1, acc = acc_val, current_loss_val = loss_val)
        if (one_train_step + 1) % 2000 == 0:
            test_data = CIFAR10_Read(test_filenames, False)
            one_test_mean_acc=[]
            one_test_mean_loss =[]
            for one_test_step in range(test_steps):
                one_test_batch_data, one_test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict={
                        x: one_test_batch_data,
                        y: one_test_batch_labels
                    }
                )
                one_test_mean_acc.append(test_acc_val)
            print_log(step = one_train_step + 1, acc = np.mean(one_test_mean_acc), is_test=True)

['../data/cifar-10-batches-py\\Data_Batch_1', '../data/cifar-10-batches-py\\Data_Batch_2', '../data/cifar-10-batches-py\\Data_Batch_3', '../data/cifar-10-batches-py\\Data_Batch_4', '../data/cifar-10-batches-py\\Data_Batch_5']
(50000, 3072) (50000,)
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1

[Train] Step 500, loss: 1.69644, acc: 0.40000.
[Train] Step 1000, loss: 1.22977, acc: 0.50000.
[Train] Step 1500, loss: 1.11991, acc: 0.55000.
[Train] Step 2000, loss: 0.97511, acc: 0.60000.
(10000, 3072) (10000,)
[Test] Step 2000, acc: 0.59400.
[Train] Step 2500, loss: 0.88426, acc: 0.70000.
[Train] Step 3000, loss: 1.05697, acc: 0.70000.
[Train] Step 3500, loss: 0.48302, acc: 0.90000.
[Train] Step 4000, loss: 0.50934, acc: 0.75000.
(10000, 3072) (10000,)
[Test] Step 4000, acc: 0.70850.
[Train] Step 4500, loss: 0.94446, acc: 0.60000.
[Train] Step 5000, loss: 0.81913, acc: 0.80000.
[Train] S